In [1]:
!pip install gensim
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 11.9 MB/s eta 0:00:00


In [2]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz

--2024-04-21 18:08:33--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45409631 (43M) [application/x-gzip]
Saving to: ‘reviews_Cell_Phones_and_Accessories_5.json.gz’

reviews_Cell_Phones 100%[===================>]  43.31M  48.6MB/s    in 0.9s    

2024-04-21 18:08:34 (48.6 MB/s) - ‘reviews_Cell_Phones_and_Accessories_5.json.gz’ saved [45409631/45409631]



In [3]:
!gunzip reviews_Cell_Phones_and_Accessories_5.json.gz

In [4]:
import gensim
import pandas as pd

In [5]:
df = pd.read_json("/content/reviews_Cell_Phones_and_Accessories_5.json",lines=True)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [6]:
df.shape

(194439, 9)

#Preprocessing & Tokenization

In [7]:
df['reviewText'][0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

In [8]:
gensim.utils.simple_preprocess("They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again")

['they',
 'look',
 'good',
 'and',
 'stick',
 'good',
 'just',
 'don',
 'like',
 'the',
 'rounded',
 'shape',
 'because',
 'was',
 'always',
 'bumping',
 'it',
 'and',
 'siri',
 'kept',
 'popping',
 'up',
 'and',
 'it',
 'was',
 'irritating',
 'just',
 'won',
 'buy',
 'product',
 'like',
 'this',
 'again']

In [9]:
review_text = df['reviewText'].apply(gensim.utils.simple_preprocess)
review_text.head()

0    [they, look, good, and, stick, good, just, don...
1    [these, stickers, work, like, the, review, say...
2    [these, are, awesome, and, make, my, phone, lo...
3    [item, arrived, in, great, time, and, was, in,...
4    [awesome, stays, on, and, looks, great, can, b...
Name: reviewText, dtype: object

#TRain Word2Vec Model

In [10]:
model = gensim.models.Word2Vec(window=10,
                               vector_size=100,
                               min_count=2,
                               sg=1) #sg=skip gram

Build The vocabulary

In [11]:
model.build_vocab(review_text)

In [12]:
model.epochs, model.window

(5, 10)

Train the word2vec model now

In [13]:
model.train(review_text, total_examples=model.corpus_count, epochs = model.epochs)

(61505629, 83868975)

In [14]:
model.corpus_count

194439

#Save the model

In [15]:
model.save("./final_word2vec.model")

find similarity words

In [16]:
# Word Embeddings
model.wv.most_similar("bad")

[('terrible', 0.7577743530273438),
 ('horrible', 0.7091153860092163),
 ('okay', 0.6659913063049316),
 ('poor', 0.6623364090919495),
 ('liike', 0.6612234115600586),
 ('good', 0.6597888469696045),
 ('reasearch', 0.6502993106842041),
 ('bleached', 0.6443766355514526),
 ('stinks', 0.6425612568855286),
 ('aweful', 0.642187237739563)]

In [17]:
model.wv.similarity(w1='cheap',w2 = 'inexpensive')

0.6368143

In [18]:
matrix = model.wv.vectors # extract the word embedding matrix

print(f"shape of the word embedding matrix is {matrix.shape}")

shape of the word embedding matrix is (35561, 100)


In [21]:
# Assuming 'model' is your Word2Vec model
W_out = model.wv.vectors
W_out.shape  # Output weights shape


(35561, 100)

In [25]:
# W_out = model.trainables.syn1neg
# W_out.shape # Output weights

In [23]:
vocab_list = list(model.wv.key_to_index.keys())

In [24]:
len(vocab_list)

35561

In [26]:
vocab_list

['the',
 'it',
 'and',
 'to',
 'is',
 'this',
 'of',
 'for',
 'my',
 'that',
 'in',
 'on',
 'phone',
 'with',
 'you',
 'case',
 'but',
 'have',
 'not',
 'was',
 'as',
 'so',
 'one',
 'very',
 'are',
 'like',
 'if',
 'be',
 'can',
 'or',
 'great',
 'your',
 'at',
 'when',
 'use',
 'screen',
 'just',
 'good',
 'all',
 'they',
 'battery',
 'from',
 'would',
 'out',
 'will',
 'well',
 'an',
 'has',
 'iphone',
 'had',
 'get',
 'charge',
 'up',
 'no',
 'me',
 'than',
 'more',
 'only',
 'charger',
 'about',
 'product',
 'other',
 'there',
 'really',
 'time',
 'also',
 'off',
 'these',
 'which',
 'works',
 'does',
 'because',
 'do',
 'don',
 'them',
 'much',
 'back',
 'what',
 'nice',
 'little',
 'price',
 'love',
 'usb',
 'its',
 'some',
 'quality',
 'charging',
 'work',
 'fit',
 'any',
 'easy',
 'even',
 've',
 'device',
 'too',
 'after',
 'still',
 'used',
 'protector',
 'while',
 'power',
 'using',
 'got',
 'better',
 'am',
 'bought',
 'two',
 'now',
 'by',
 'cable',
 'first',
 'recommend'

In [28]:
vocab_len = len(vocab_list)
vocab = model.wv.key_to_index

In [29]:
vocab

{'the': 0,
 'it': 1,
 'and': 2,
 'to': 3,
 'is': 4,
 'this': 5,
 'of': 6,
 'for': 7,
 'my': 8,
 'that': 9,
 'in': 10,
 'on': 11,
 'phone': 12,
 'with': 13,
 'you': 14,
 'case': 15,
 'but': 16,
 'have': 17,
 'not': 18,
 'was': 19,
 'as': 20,
 'so': 21,
 'one': 22,
 'very': 23,
 'are': 24,
 'like': 25,
 'if': 26,
 'be': 27,
 'can': 28,
 'or': 29,
 'great': 30,
 'your': 31,
 'at': 32,
 'when': 33,
 'use': 34,
 'screen': 35,
 'just': 36,
 'good': 37,
 'all': 38,
 'they': 39,
 'battery': 40,
 'from': 41,
 'would': 42,
 'out': 43,
 'will': 44,
 'well': 45,
 'an': 46,
 'has': 47,
 'iphone': 48,
 'had': 49,
 'get': 50,
 'charge': 51,
 'up': 52,
 'no': 53,
 'me': 54,
 'than': 55,
 'more': 56,
 'only': 57,
 'charger': 58,
 'about': 59,
 'product': 60,
 'other': 61,
 'there': 62,
 'really': 63,
 'time': 64,
 'also': 65,
 'off': 66,
 'these': 67,
 'which': 68,
 'works': 69,
 'does': 70,
 'because': 71,
 'do': 72,
 'don': 73,
 'them': 74,
 'much': 75,
 'back': 76,
 'what': 77,
 'nice': 78,
 'little

In [31]:
vocab['sum']

4711

In [34]:
vocab_int2word = {c.index:w for w,c in vocab.items()}

AttributeError: 'int' object has no attribute 'index'

In [36]:
# Assuming 'vocab' is a dictionary of word-to-count mappings
vocab_int2word = {idx: word for word, idx in vocab.items()}


In [37]:
vocab_int2word

{0: 'the',
 1: 'it',
 2: 'and',
 3: 'to',
 4: 'is',
 5: 'this',
 6: 'of',
 7: 'for',
 8: 'my',
 9: 'that',
 10: 'in',
 11: 'on',
 12: 'phone',
 13: 'with',
 14: 'you',
 15: 'case',
 16: 'but',
 17: 'have',
 18: 'not',
 19: 'was',
 20: 'as',
 21: 'so',
 22: 'one',
 23: 'very',
 24: 'are',
 25: 'like',
 26: 'if',
 27: 'be',
 28: 'can',
 29: 'or',
 30: 'great',
 31: 'your',
 32: 'at',
 33: 'when',
 34: 'use',
 35: 'screen',
 36: 'just',
 37: 'good',
 38: 'all',
 39: 'they',
 40: 'battery',
 41: 'from',
 42: 'would',
 43: 'out',
 44: 'will',
 45: 'well',
 46: 'an',
 47: 'has',
 48: 'iphone',
 49: 'had',
 50: 'get',
 51: 'charge',
 52: 'up',
 53: 'no',
 54: 'me',
 55: 'than',
 56: 'more',
 57: 'only',
 58: 'charger',
 59: 'about',
 60: 'product',
 61: 'other',
 62: 'there',
 63: 'really',
 64: 'time',
 65: 'also',
 66: 'off',
 67: 'these',
 68: 'which',
 69: 'works',
 70: 'does',
 71: 'because',
 72: 'do',
 73: 'don',
 74: 'them',
 75: 'much',
 76: 'back',
 77: 'what',
 78: 'nice',
 79: 'li

In [38]:
vocab_int2word[4697]

'nit'

In [39]:
import numpy as np
one_hot = np.zeros(shape=(1,vocab_len))

one_hot[0,vocab['awesome'].index] = 1

AttributeError: 'int' object has no attribute 'index'

In [40]:
# Assuming 'vocab' is a dictionary of word-to-index mappings
vocab_len = len(vocab)
one_hot = np.zeros(shape=(1, vocab_len))

# Check if 'awesome' is in vocab before accessing its index
if 'awesome' in vocab:
    index_of_awesome = vocab['awesome']
    one_hot[0, index_of_awesome] = 1
else:
    print("'awesome' is not in the vocabulary")

# Display the one-hot encoding
print(one_hot)


[[0. 0. 0. ... 0. 0. 0.]]


In [41]:
z_proj = np.dot(one_hot,matrix)
z_proj.shape

(1, 100)

In [ ]:
a_proj = z_proj

z_out = np.dot(a_proj,W_out.T)
z_out.shape

In [43]:
#Softmax
a_out = np.exp(z_out)/np.sum(np.exp(z_out))

In [44]:
a_out.shape

(1, 35561)

In [45]:
a_out = a_out.flatten()

indices = np.argsort(a_out) #sort in ascending order
indices = indices[::-1] # sort in descending order

for i in range(10):
  word_index = indices[i]
  word = vocab_int2word[word_index]
  a_out_i = a_out[word_index]

  print(f"{word} with score {a_out_i}")

awesome with score 0.0007199690924412073
amazing with score 0.0006120352236078737
awsome with score 0.0005883203885462291
unbeatable with score 0.0004943999682087525
luv with score 0.00041995588741759217
vivid with score 0.00041395081943751514
goodry with score 0.0003693849485291217
wonderful with score 0.00034505308655080494
outstanding with score 0.000345013063668794
excelent with score 0.0003425202254092329


#Continious bag of words (CBOW) model

In [46]:
model = gensim.models.Word2Vec(window=10,
                               vector_size=100,
                               min_count=2,
                               sg=0) #When we set sg=0, it means we use CBOW

In [47]:
model.build_vocab(review_text)

In [48]:
model.epochs

5

In [49]:
model.train(review_text, total_examples=model.corpus_count, epochs = model.epochs)

(61501707, 83868975)

In [ ]:
model.corpus_count

In [51]:
model.save("./final_word2veccbow.model")

In [52]:
# Word Embeddings
model.wv.most_similar("bad")

[('terrible', 0.6679076552391052),
 ('shabby', 0.6474851965904236),
 ('good', 0.5998085141181946),
 ('horrible', 0.5984801054000854),
 ('legit', 0.5800608992576599),
 ('awful', 0.5620303153991699),
 ('crappy', 0.5402722358703613),
 ('disappointing', 0.5175734162330627),
 ('poor', 0.5171399712562561),
 ('okay', 0.5081954598426819)]